In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and prepare the dataset
file_path = 'goal_ticks.csv'
column_names = [
    "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "dist_to_ball", "kart_X", "kart_Z", 
    "vel_X", "vel_Z", "speed", "steer", "accel", "brake", "skid", "goal"
]

try:
    df = pd.read_csv(file_path, header=None, names=column_names)
except Exception as e:
    print(f"Error loading the data file: {e}")
    exit(1)

# Extract features and labels
X = df.iloc[:, :-1].values  # All columns except the last one
y = df['goal'].values  # The 'goal' column as binary

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Neural Network Architecture for Scoring Model
class ScoringModel(nn.Module):
    def __init__(self, num_inputs):
        super(ScoringModel, self).__init__()
        self.fc1 = nn.Linear(num_inputs, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)  # Outputting a single score for goal likelihood

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Sigmoid activation to output a probability
        return x

# Model initialization
model = ScoringModel(X_tensor.shape[1])

# Loss and optimizer for a binary classification problem
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Split data, create loaders, train and validate
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training and validation functions
def train_model(train_loader, model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))  # Ensure labels are the correct shape
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

def validate_model(val_loader, model, criterion):
    model.eval()
    total_loss = 0
    count = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            total_loss += loss.item()
            count += 1
    avg_loss = total_loss / count
    print(f'Validation Loss: {avg_loss:.4f}')

# Execute training and validation
train_model(train_loader, model, criterion, optimizer, 20)
validate_model(val_loader, model, criterion)

# Save the trained model
model.eval()
example_input = torch.rand(1, X_train.shape[1])  # Generate a random example input
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("value_based_model.pt")


Epoch 1/20, Loss: 0.1647
Epoch 2/20, Loss: 0.1047
Epoch 3/20, Loss: 0.0892
Epoch 4/20, Loss: 0.1498
Epoch 5/20, Loss: 0.0539
Epoch 6/20, Loss: 0.0390
Epoch 7/20, Loss: 0.0806
Epoch 8/20, Loss: 0.0508
Epoch 9/20, Loss: 0.0803
Epoch 10/20, Loss: 0.0691
Epoch 11/20, Loss: 0.0707
Epoch 12/20, Loss: 0.0502
Epoch 13/20, Loss: 0.0842
Epoch 14/20, Loss: 0.0787
Epoch 15/20, Loss: 0.0983
Epoch 16/20, Loss: 0.0724
Epoch 17/20, Loss: 0.1646
Epoch 18/20, Loss: 0.1248
Epoch 19/20, Loss: 0.0479
Epoch 20/20, Loss: 0.1169
Validation Loss: 0.1214
